## Setup

In [1]:
df = read.csv('../data/cleaned.csv', check.names=FALSE)
df

AfterProbiotic,(12 or 13)-methylmyristate (a15:0 or i15:0),(14 or 15)-methylpalmitate (a17:0 or i17:0),(16 or 17)-methylstearate (a19:0 or i19:0),(3'-5')-adenylyladenosine*,(3'-5')-adenylylcytidine,(3'-5')-adenylyluridine,(3'-5')-cytidylylcytidine*,(3'-5')-cytidylylguanosine,(3'-5')-cytidylyluridine*,...,sedoheptulose,suberate (C8-DC),succinate,succinimide,threonate,tricarballylate,urate,xylose,mouseID,isMale
0,0.09769943,0.03126430,0.01175118,0.0025300513,0.0015953406,0.0025319685,0.0026998886,0.0015221590,0.0026410562,...,0.04072892,0.02473527,0.3473332,0.02198092,0.15567713,0.24143204,0.04836660,0.14906799,1,1
1,0.15894081,0.03345032,0.01087362,0.0006593277,0.0006593277,0.0006593277,0.0006593277,0.0006593277,0.0012362812,...,0.03418842,0.04274254,0.3746433,0.03385185,0.08829207,0.07861127,0.04443141,0.18018382,1,1
0,0.11959562,0.03428829,0.01158458,0.0014854743,0.0007846236,0.0016378315,0.0018177270,0.0007846236,0.0019728938,...,0.06084485,0.03310305,0.2866504,0.03065286,0.11693983,0.19700936,0.04518001,0.16832822,2,1
1,0.13091614,0.03704059,0.01544693,0.0009213574,0.0009213574,0.0009213574,0.0009213574,0.0009213574,0.0009213574,...,0.09177728,0.03599431,0.4429625,0.02220815,0.07813900,0.10542181,0.03251015,0.14004571,2,1
0,0.10378773,0.03746011,0.01454331,0.0025258186,0.0024364777,0.0027643357,0.0035661809,0.0007091002,0.0033885674,...,0.03437333,0.03238493,0.2864540,0.02873003,0.19599581,0.32846756,0.05767700,0.07274028,3,1
1,0.15449811,0.03694344,0.01405253,0.0033453494,0.0022560673,0.0035784999,0.0034529102,0.0023853646,0.0035252966,...,0.05892596,0.04487653,0.3616701,0.03120959,0.12471296,0.13902043,0.06309067,0.09322475,3,1
0,0.17824911,0.04016364,0.01589867,0.0030008185,0.0022350649,0.0031025425,0.0030461414,0.0009414246,0.0031171519,...,0.02181575,0.03059914,0.3447633,0.02945386,0.15395690,0.25346776,0.06358654,0.11890044,4,0
1,0.11162569,0.03075153,0.01109551,0.0008319971,0.0008319971,0.0008319971,0.0008319971,0.0008319971,0.0008319971,...,0.02158385,0.03351839,0.2456366,0.02058728,0.18538022,0.15985092,0.08296148,0.10155370,4,0
0,0.16020749,0.03896578,0.01305617,0.0028324050,0.0018931417,0.0032457082,0.0009337653,0.0009337653,0.0023826135,...,0.02633124,0.02768154,0.2404378,0.02345896,0.12133600,0.19704572,0.05242202,0.28736344,5,0
1,0.12925316,0.03119683,0.01179281,0.0010613359,0.0010613359,0.0010613359,0.0034758482,0.0010613359,0.0035674905,...,0.02390423,0.03044215,0.2163461,0.02434295,0.19916877,0.17463974,0.08163638,0.05505979,5,0


In [2]:
biochemicals = colnames(df)[2:824]
head(biochemicals)

[1] "(12 or 13)-methylmyristate (a15:0 or i15:0)"
[2] "(14 or 15)-methylpalmitate (a17:0 or i17:0)"
[3] "(16 or 17)-methylstearate (a19:0 or i19:0)" 
[4] "(3'-5')-adenylyladenosine*"                 
[5] "(3'-5')-adenylylcytidine"                   
[6] "(3'-5')-adenylyluridine"

## Setup helper functions for each model

#### Model 1: dependent variable is metabolite; independent variable is time-point.


In [3]:
get_p_val_m1 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX)]
    colnames(data)[2] = 'abundance'
    model = lm(abundance ~ AfterProbiotic, data=data, family=binomial)
    coef = coef(summary(model))[,4]
    p_val = coef[2]
    return (p_val)
}

In [4]:
# Test function on first biochemical
get_p_val_m1(df, 2)

Warning message:
“In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument ‘family’ will be disregarded”

AfterProbiotic 
     0.7842764

#### Model 2: dependent variable is metabolite; independent variables are time-point and mouse sex.

In [5]:
get_p_val_m2 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX, 826)]
    colnames(data)[2] = 'abundance'
    model = lm(abundance ~ AfterProbiotic + isMale, data=data, family=binomial)
    coef = coef(summary(model))[,4]
    p_val_time = coef[2]
    return (p_val_time)
}

In [6]:
# Test function on first biochemical
get_p_val_m2(df, 2)

Warning message:
“In lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) :
 extra argument ‘family’ will be disregarded”

AfterProbiotic 
     0.7872136

#### Model 3: dependent variable is metabolite; independent variables are time-point and mouse sex, and also control for mouseID as a random-effect (use 'lmer' function with (1|mouseID).

In [ ]:
library(lmerTest)
get_p_val_m3 = function(dataframe, biochemIDX) {
    # dataframe is the dataframe with all the data.
    # biochemIDX is the column number of the metabolite used as the dependent var
    # returns the p-value associated with the coefficient for the time point variable
    data = dataframe[, c(1, biochemIDX, 825, 826)]
    colnames(data)[2] = 'abundance'
    model = lmer(abundance ~ AfterProbiotic + isMale + (1|mouseID), data=data)
    coef = coef(summary(model))[,5]
    p_val_time = coef[2]
    return (p_val_time)
}

In [9]:
# Test function on first biochemical
get_p_val_m3(df, 2)

boundary (singular) fit: see ?isSingular


AfterProbiotic 
     0.7872136

In [10]:
get_p_vals = function(dataframe, model_func) {
    # dataframe is the dataframe with all the data.
    # model_func is the function that returns the p-values for the model of interest
    # returns the p-values associated with the time variable for each metabolite
    p_vals = data.frame(
        biochemical = biochemicals,
        p_val = -1
    )
    print(head(p_vals))
    for (biochemIDX in 2:824) {
        p_val = tryCatch(
          model_func(dataframe, biochemIDX),
          error=function(e) {return (1)}
        )
        p_vals[biochemIDX - 1, 2] = p_val
    }
    return (p_vals)
}

## Run the three models on the 823 metabolites

In [ ]:
p_vals_m1 = get_p_vals(df, get_p_val_m1)
p_vals_m2 = get_p_vals(df, get_p_val_m2)
p_vals_m3 = get_p_vals(df, get_p_val_m3)

In [12]:
head(p_vals_m3)

biochemical,p_val
(12 or 13)-methylmyristate (a15:0 or i15:0),0.7872136
(14 or 15)-methylpalmitate (a17:0 or i17:0),0.2954750
(16 or 17)-methylstearate (a19:0 or i19:0),0.6007737
(3'-5')-adenylyladenosine*,0.1158123
(3'-5')-adenylylcytidine,0.0801481
(3'-5')-adenylyluridine,0.1002768


## Analysis

In [13]:
p_vals_m3_adjusted = data.frame(p_vals_m3)
p_vals_m3_adjusted[, 2] = p.adjust(p_vals_m3_adjusted[,2], method='BH')
head(p_vals_m3_adjusted)

biochemical,p_val
(12 or 13)-methylmyristate (a15:0 or i15:0),0.9357310
(14 or 15)-methylpalmitate (a17:0 or i17:0),0.6064238
(16 or 17)-methylstearate (a19:0 or i19:0),0.8539495
(3'-5')-adenylyladenosine*,0.3841295
(3'-5')-adenylylcytidine,0.3381292
(3'-5')-adenylyluridine,0.3635794


In [14]:
p_vals_m3_sig = p_vals_m3_adjusted[p_vals_m3_adjusted[, 'p_val'] <= 0.20,]
ordered = p_vals_m3_sig[order(p_vals_m3_sig$p_val),]
print(nrow(ordered))
head(ordered, 15)

[1] 118


,biochemical,p_val
344,5-methyl-2'-deoxycytidine,0.02028685
184,glycocholate sulfate,0.12446353
222,myristoleate (14:1n5),0.12446353
473,N-acetyl-3-methylhistidine*,0.12446353
696,palmitoleoyl ethanolamide*,0.12446353
710,pheophytin A,0.12446353
23,1-carboxyethylisoleucine,0.13974845
143,citrate,0.13974845
149,deoxycholate,0.13974845
217,lithocholate,0.13974845


In [14]:
write.csv(ordered, '../output/p_vals_m3.csv', row.names=FALSE)

## Calculate pre/post means for significant metabolites

In [20]:
pre = subset(df, AfterProbiotic == 0)[, 2:824]
pre = subset(pre, select=biochemicals_order)
print(ncol(pre))
pre_means = as.vector(colMeans(pre))
pre_means[1:5]

[1] 823


[1] 0.006088053 0.003140369 0.034112072 0.013613349 0.031938988

In [21]:
post = subset(df, AfterProbiotic == 1)[, 2:824]
post = subset(post, select=biochemicals_order)
print(ncol(post))
post_means = as.vector(colMeans(post))
post_means[1:5]

[1] 823


[1] 0.008384498 0.001202254 0.027995221 0.018672667 0.020593015

In [22]:
means = data.frame(
    means_pre = pre_means,
    means_post = post_means
)
output = cbind(ordered_all, means)
colnames(output) = c("Biochemical Name", "adj-P-value (BH)", "mean (Pre)", "mean (Post)")
head(output)

,Biochemical Name,adj-P-value (BH),mean (Pre),mean (Post)
344,5-methyl-2'-deoxycytidine,0.02028685,0.006088053,0.008384498
184,glycocholate sulfate,0.12446353,0.003140369,0.001202254
222,myristoleate (14:1n5),0.12446353,0.034112072,0.027995221
473,N-acetyl-3-methylhistidine*,0.12446353,0.013613349,0.018672667
696,palmitoleoyl ethanolamide*,0.12446353,0.031938988,0.020593015
710,pheophytin A,0.12446353,0.055106928,0.033595361


In [23]:
write.csv(output, '../output/p_vals_all_means.csv', row.names=FALSE)